<a href="https://colab.research.google.com/github/wjsrlzhem/physiognomy/blob/main/physiognomy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import matplotlib
matplotlib.use("Agg")

from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import random
import pickle
import cv2
import os


In [12]:
ap = argparse.ArgumentParser()
ap.add_argument("-d","--dataset",help = "path to dataset")
ap.add_argument("-m","--model",help = "path to ")
ap.add_argument("-l","--label-bin",help = "path to")
ap.add_argument("-p","--plot",help="path to")

#!!에러
#args = ap.parse_args()

args,_ = ap.parse_known_args()
args = vars(args)
args

{'dataset': None, 'label_bin': None, 'model': None, 'plot': None}

In [10]:
print("[INFO] loading images...")
data = []
labels = []

imagePaths= sorted(list(paths.list_images(args['dataset'])))
random.seed(42)
random.shuffle(imagePaths)

for imagePath in imagePaths:
  image = cv2.imread(imagePath)
  image = cv2.resize(image,(32,32)).flatten()
  data.append(image)

  label = imagePath.split(os.path.sep)[-2]
  labels.append(label)
  

[INFO] loading images...


TypeError: ignored

In [ ]:
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)

In [ ]:
(trainX,testX,trainY,testY) = train_test_split(data,labels, test_size= 0.25, random_state = 42)


In [ ]:
lb = LabelBinarizer()
trainY = lb.fit_transform(trainY)
testY = lb.transform(testY)

In [ ]:
model = Sequential()
model.add(Dense(1024,input_shape = (3072,),activation='sigmoid'))
model.add(Dense(512,activation = 'sigmoid'))
model.add(Dense(len(lb.classes_),activation = 'softmax'))

In [ ]:
INIT_LR = 0.01
EPOCHS= 80

print("[INFO] training network ...")
opt = SGD(lr = INIT_LR)
model.compile(loss='categorical_crossentropy',optimizer = opt,
              metrics = ['accuracy'])


In [ ]:
H = model.fit(x=trainX,y=trainY,validation_data =(testX,testY),
              epochs=EPOCHS,batch_size=32)

In [ ]:
print('[INFO] evaluationg network...')
predictions = model.predict(x=testX,batch_size=32)
print(classification_report(testY.argmax(axis=1),
                            predictions.argmax(axis=1),traget_names=lb.classes_))

N = np.arrange(0,EPOCHS)
plt.style.use('ggplot')
plt.figure()
plt.plot(N,H.history['loss'],label='train_loss')
plt.plot(N,H.history['val_loss'],label='val_loss')
plt.plot(N,H.history['accuracy'],label='train_acc')
plt.plot(N,H.history['val_accuracy'],label='val_acc')

plt.title("Training Loss and Accuracy (Simple NN")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend()
plt.savefig(args['plot'])